In [ ]:
notefrom google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
# !ls

--2022-02-22 10:50:55--  https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 300971569 (287M) [application/x-gzip]
Saving to: ‘spark-3.2.1-bin-hadoop3.2.tgz’

spark-3.2.1-bin-had 100%[===================>] 287.03M   265MB/s    in 1.1s    

2022-02-22 10:50:57 (265 MB/s) - ‘spark-3.2.1-bin-hadoop3.2.tgz’ saved [300971569/300971569]



In [ ]:
!tar -xzf /content/spark-3.2.1-bin-hadoop3.2.tgz
# !ls 

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [ ]:
import findspark

In [ ]:
findspark.init("/content/spark-3.2.1-bin-hadoop3.2/")

In [ ]:
findspark.find()

'/content/spark-3.2.1-bin-hadoop3.2/'

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark= SparkSession \
       .builder \
       .appName("Our First Spark example") \
       .getOrCreate()

In [ ]:
spark

In [ ]:
df = spark.read.load("/content/sample_data/california_housing_train.csv", format='csv', header=True)

In [ ]:
df.limit(1).withColumn("address", df.latitude).toJSON(['UTF-8']).collect()

['{"longitude":"-114.310000","latitude":"34.190000","housing_median_age":"15.000000","total_rooms":"5612.000000","total_bedrooms":"1283.000000","population":"1015.000000","households":"472.000000","median_income":"1.493600","median_house_value":"66900.000000","address":"34.190000"}']

In [ ]:
@udf(returnType=MapType(StringType(), StringType()))
def json_struct_type(cols):
  return {"longitude": cols.longitude, "latitude": cols.latitude}

In [ ]:
df.withColumn("value", json_struct_type(struct(df.longitude, df.latitude))).toJSON().collect()

In [ ]:
def list_type(cols):
  return [cols.longitude, cols.latitude]
@udf(returnType=ArrayType(StringType()))
def json_list_type(cols):
  return list_type(cols)

In [ ]:
df.withColumn("value", json_list_type(struct(df.longitude, df.latitude))).toJSON().first()

'{"longitude":"-114.310000","latitude":"34.190000","housing_median_age":"15.000000","total_rooms":"5612.000000","total_bedrooms":"1283.000000","population":"1015.000000","households":"472.000000","median_income":"1.493600","median_house_value":"66900.000000","value":["-114.310000","34.190000"]}'

In [ ]:
schema = ArrayType( MapType(StringType(), StringType()))

@udf(returnType=schema)
def json_dict_type(cols):
  return [{"longitude": cols.longitude, "latitude": cols.latitude, "housing_median_age": cols.housing_median_age}, {"longitude": cols.longitude, "latitude": cols.latitude}]

In [ ]:
df.select("housing_median_age", json_dict_type(struct(df.longitude, df.latitude)).alias("locate")).toJSON().first()

'{"housing_median_age":"15.000000","locate":[{"latitude":"34.190000","longitude":"-114.310000"},{"latitude":"34.190000","longitude":"-114.310000"}]}'

In [ ]:
df.withColumn("version", lit("11111")).printSchema()

In [ ]:
schema = ArrayType( MapType(StringType(), StringType()))

@udf(returnType=schema)
def json_dict_type(cols):
  return [
          {"a": cols[0].longitude}
        ]

c = [["longitude", "latitude", "housing_median_age"], ["longitude", "latitude", "housing_median_age"]]
columns = struct(struct(c[0]), struct(c[1]))
df.select("housing_median_age", json_dict_type(columns).alias("locate")).toJSON().first()

'{"housing_median_age":"15.000000","locate":[{"a":"-114.310000"}]}'

In [ ]:
import json
schema = StringType()

@udf(returnType=schema)
def json_dict_type(cols):

  val = []
  for i in cols:
    val.append(
            {"a": i.longitude}
          )
  return str(val)

c = [["longitude", "latitude", "housing_median_age"], ["longitude", "latitude", "housing_median_age"]]
columns = struct(struct(c[0]), struct(c[1]))
df.select("housing_median_age", json_dict_type(columns).alias("locate")).toJSON().first()

'{"housing_median_age":"15.000000","locate":"[{\'a\': \'-114.310000\'}, {\'a\': \'-114.310000\'}]"}'

In [ ]:
schema = ArrayType( MapType(StringType(), StringType()))

@udf(returnType=schema)
def json_dict_type(cols):

  val = []
  for i in cols:
    val.append(
            {"a": i.longitude}
          )
  return val

c = [["longitude", "latitude", "housing_median_age"], ["longitude", "latitude", "housing_median_age"]]
columns = struct(struct(c[0]), struct(c[1]))
df.select("housing_median_age", json_dict_type(columns).alias("locate")).toJSON().first()

In [ ]:
a = [{'a':1},{'a':1}]
print(str(a))

[{'a': 1}, {'a': 1}]


In [ ]:
df1 = spark.read.load("/content/sample_data/california_housing_train.csv", format='csv', header=True).withColumn("idx", monotonically_increasing_id())

In [ ]:
df2 = spark.read.load("/content/sample_data/california_housing_test.csv", format='csv', header=True).withColumn("idx", monotonically_increasing_id())

In [ ]:
# {
#     "table1": tab1,
#     "table2": tab2,
#     "table3": tab3
# }

In [ ]:
dd = {
    "t1": df1.toJSON().collect(),
    "t2": df2.toJSON().collect()
}

In [ ]:
dd

{'t1': ['{"longitude":"-114.310000","latitude":"34.190000","housing_median_age":"15.000000","total_rooms":"5612.000000","total_bedrooms":"1283.000000","population":"1015.000000","households":"472.000000","median_income":"1.493600","median_house_value":"66900.000000","idx":0}',
  '{"longitude":"-114.470000","latitude":"34.400000","housing_median_age":"19.000000","total_rooms":"7650.000000","total_bedrooms":"1901.000000","population":"1129.000000","households":"463.000000","median_income":"1.820000","median_house_value":"80100.000000","idx":1}',
  '{"longitude":"-114.560000","latitude":"33.690000","housing_median_age":"17.000000","total_rooms":"720.000000","total_bedrooms":"174.000000","population":"333.000000","households":"117.000000","median_income":"1.650900","median_house_value":"85700.000000","idx":2}',
  '{"longitude":"-114.570000","latitude":"33.640000","housing_median_age":"14.000000","total_rooms":"1501.000000","total_bedrooms":"337.000000","population":"515.000000","households